# Farmer problem

Reference: Birge and Louveaux, "Introduction to Stochastic Programming", Chapter 1

In [3]:
using JuMP

In [4]:
# Open-source solvers
using GLPK
using HiGHS

Commercial solvers as `CPLEX` or `Gurobi` can also be used.

Among open-source solvers, we recommend HiGHS ((https://highs.info/), as suggested by the presentation done by Julian Hall at JuliaCon 2022: https://www.youtube.com/watch?v=ChKdTqUzXdY

# Average Scenario

We construct the model for the average scenario, using first the open-source HiGHS solver .

In [5]:
m = Model(HiGHS.Optimizer)
#m = Model(GLPK.Optimizer)

crops = ["Wheat", "Corn", "Sugar beets"]
ncrops = length(crops)

@variable(m, x[1:ncrops] >= 0)
@variable(m, y[1:2] >= 0)
@variable(m, w[1:4] >= 0)

costSeeding = [150, 230, 260]
costs = [238, 210]
prices = [170, 150, 36, 10]

4-element Vector{Int64}:
 170
 150
  36
  10

We can check that `m` is a JuMP model by checking its type.

In [6]:
typeof(m)

Model (alias for GenericModel{Float64})

In [7]:
typeof(x)

Vector{VariableRef} (alias for Array{GenericVariableRef{Float64}, 1})

We now add the constraints.

In [8]:
@constraint(m, surface, sum(x[i] for i=1:ncrops) <= 500)

surface : x[1] + x[2] + x[3] <= 500

In [9]:
@constraint(m, wheatNeeds, 2.5x[1]+y[1]-w[1] >= 200)

wheatNeeds : 2.5 x[1] + y[1] - w[1] >= 200

In [10]:
@constraint(m, cornNeeds, 3x[2]+y[2]-w[2] >= 240)

cornNeeds : 3 x[2] + y[2] - w[2] >= 240

In [11]:
@constraint(m, beetsProd, w[3]+w[4] <= 20x[3])

beetsProd : -20 x[3] + w[3] + w[4] <= 0

In [12]:
@constraint(m, beetsQuota, w[3] <= 6000)

beetsQuota : w[3] <= 6000

Let's define the objective function. 

In [13]:
@objective(m, Min, sum(costSeeding[i]*x[i] for i=1:ncrops) + sum(costs[i]*y[i] for i = 1:length(costs))
    - (sum(prices[i]*w[i] for i=1:length(prices))))

150 x[1] + 230 x[2] + 260 x[3] + 238 y[1] + 210 y[2] - 170 w[1] - 150 w[2] - 36 w[3] - 10 w[4]

The complete model can be displayed with the command `print`.

In [14]:
print(m)

More information can be obtained with the method `show`:

In [15]:
show(m)

A JuMP Model
├ solver: HiGHS
├ objective_sense: MIN_SENSE
│ └ objective_function_type: AffExpr
├ num_variables: 9
├ num_constraints: 14
│ ├ AffExpr in MOI.GreaterThan{Float64}: 2
│ ├ AffExpr in MOI.LessThan{Float64}: 3
│ └ VariableRef in MOI.GreaterThan{Float64}: 9
└ Names registered in the model
  └ :beetsProd, :beetsQuota, :cornNeeds, :surface, :w, :wheatNeeds, :x, :y

We now solve it.

In [16]:
optimize!(m)

Running HiGHS 1.11.0 (git hash: 364c83a51e): Copyright (c) 2025 HiGHS under MIT licence terms
LP   has 5 rows; 9 cols; 13 nonzeros
Coefficient ranges:
  Matrix [1e+00, 2e+01]
  Cost   [1e+01, 3e+02]
  Bound  [0e+00, 0e+00]
  RHS    [2e+02, 6e+03]
Presolving model
4 rows, 9 cols, 12 nonzeros  0s
3 rows, 8 cols, 10 nonzeros  0s
Dependent equations search running on 2 equations with time limit of 1000.00s
Dependent equations search removed 0 rows and 0 nonzeros in 0.00s (limit = 1000.00s)
3 rows, 7 cols, 9 nonzeros  0s
Presolve : Reductions: rows 3(-2); columns 7(-2); elements 9(-4)
Solving the presolved LP
Using EKK dual simplex solver - serial
  Iteration        Objective     Infeasibilities num(sum)
          0    -6.3999740210e+02 Ph1: 2(2); Du: 2(639.997) 0s
          4    -1.1860000000e+05 Pr: 0(0) 0s
Solving the original LP from the solution after postsolve
Model status        : Optimal
Simplex   iterations: 4
Objective value     : -1.1860000000e+05
P-D objective error :  0.0000000

In [17]:
print("Solution: $(value.(x))")

Solution: [120.0, 80.0, 300.0]

In [18]:
? value

search: value values valtype set_value fix_value falses value_type evalfile



```
value(con_ref::ConstraintRef; result::Int = 1)
```

Return the primal value of constraint `con_ref` associated with result index `result` of the most-recent solution returned by the solver.

That is, if `con_ref` is the reference of a constraint `func`-in-`set`, it returns the value of `func` evaluated at the value of the variables (given by [`value(::GenericVariableRef)`](@ref)).

Use [`primal_status`](@ref) to check if a result exists before asking for values.

See also: [`result_count`](@ref).

## Note

For scalar constraints, the constant is moved to the `set` so it is not taken into account in the primal value of the constraint. For instance, the constraint `@constraint(model, 2x + 3y + 1 == 5)` is transformed into `2x + 3y`-in-`MOI.EqualTo(4)` so the value returned by this function is the evaluation of `2x + 3y`.

---

```
value(var_value::Function, con_ref::ConstraintRef)
```

Evaluate the primal value of the constraint `con_ref` using `var_value(v)` as the value for each variable `v`.

---

```
value(v::GenericVariableRef; result = 1)
```

Return the value of variable `v` associated with result index `result` of the most-recent returned by the solver.

Use [`primal_status`](@ref) to check if a result exists before asking for values.

See also: [`result_count`](@ref).

---

```
value(var_value::Function, v::GenericVariableRef)
```

Evaluate the value of the variable `v` as `var_value(v)`.

---

```
value(var_value::Function, ex::GenericAffExpr)
```

Evaluate `ex` using `var_value(v)` as the value for each variable `v`.

---

```
value(v::GenericAffExpr; result::Int = 1)
```

Return the value of the `GenericAffExpr` `v` associated with result index `result` of the most-recent solution returned by the solver.

See also: [`result_count`](@ref).

---

```
value(var_value::Function, ex::GenericQuadExpr)
```

Evaluate `ex` using `var_value(v)` as the value for each variable `v`.

---

```
value(v::GenericQuadExpr; result::Int = 1)
```

Return the value of the `GenericQuadExpr` `v` associated with result index `result` of the most-recent solution returned by the solver.

Replaces `getvalue` for most use cases.

See also: [`result_count`](@ref).

---

```
value(p::NonlinearParameter)
```

Return the current value stored in the nonlinear parameter `p`.

## Example

```jldoctest
julia> model = Model();

julia> @NLparameter(model, p == 10)
p == 10.0

julia> value(p)
10.0
```

---

```
value(ex::NonlinearExpression; result::Int = 1)
```

Return the value of the `NonlinearExpression` `ex` associated with result index `result` of the most-recent solution returned by the solver.

See also: [`result_count`](@ref).

---

```
value(var_value::Function, ex::NonlinearExpression)
```

Evaluate `ex` using `var_value(v)` as the value for each variable `v`.

---

```
value(c::NonlinearConstraintRef; result::Int = 1)
```

Return the value of the `NonlinearConstraintRef` `c` associated with result index `result` of the most-recent solution returned by the solver.

See also: [`result_count`](@ref).

---

```
value(var_value::Function, c::NonlinearConstraintRef)
```

Evaluate `c` using `var_value(v)` as the value for each variable `v`.


In [19]:
methods(value)

# 24 methods for generic function "value" from JuMP:
  [1] value(ex::NonlinearExpression; result)
     @ C:\Users\slash\.julia\packages\JuMP\Ue9wj\src\nlp.jl:413
  [2] value(::MutableArithmetics.Zero; result)
     @ C:\Users\slash\.julia\packages\JuMP\Ue9wj\src\variables.jl:2710
  [3] value(p::NonlinearParameter)
     @ C:\Users\slash\.julia\packages\JuMP\Ue9wj\src\nlp.jl:285
  [4] value(Q::LinearAlgebra.Symmetric{V, Matrix{V}}; result) where V<:AbstractJuMPScalar
     @ C:\Users\slash\.julia\packages\JuMP\Ue9wj\src\sd.jl:867
  [5] value(var_value::Function, ex::NonlinearExpression)
     @ C:\Users\slash\.julia\packages\JuMP\Ue9wj\src\nlp.jl:448
  [6] value(var_value::Function, Q::LinearAlgebra.Symmetric{V, Matrix{V}}) where V<:AbstractJuMPScalar
     @ C:\Users\slash\.julia\packages\JuMP\Ue9wj\src\sd.jl:857
  [7] value(::Function, x::Number)
     @ C:\Users\slash\.julia\packages\JuMP\Ue9wj\src\variables.jl:2714
  [8] value(var_value::Function, c::NonlinearConstraintRef)
     @ C:\Users\slash\.julia\packages\JuMP\Ue9wj\src\nlp.jl:630
  [9] value(var_value::Function, con_ref::ConstraintRef{<:AbstractModel, <:MathOptInterface.ConstraintIndex})
     @ C:\Users\slash\.julia\packages\JuMP\Ue9wj\src\constraints.jl:1315
 [10] value(var_value::Function, Q::LinearAlgebra.Hermitian{V, Matrix{V}}) where V<:AbstractJuMPScalar
     @ C:\Users\slash\.julia\packages\JuMP\Ue9wj\src\sd.jl:877
 [11] value(var_value::Function, ex::GenericQuadExpr{CoefType, VarType}) where {CoefType, VarType}
     @ C:\Users\slash\.julia\packages\JuMP\Ue9wj\src\quad_expr.jl:843
 [12] value(var_value::Function, v::GenericVariableRef)
     @ C:\Users\slash\.julia\packages\JuMP\Ue9wj\src\variables.jl:2100
 [13] value(var_value::Function, Q::AbstractArray{<:AbstractJuMPScalar})
     @ C:\Users\slash\.julia\packages\JuMP\Ue9wj\src\variables.jl:2701
 [14] value(f::Function, expr::GenericNonlinearExpr)
     @ C:\Users\slash\.julia\packages\JuMP\Ue9wj\src\nlp_expr.jl:652
 [15] value(var_value::Function, ex::GenericAffExpr{T, V}) where {T, V}
     @ C:\Users\slash\.julia\packages\JuMP\Ue9wj\src\aff_expr.jl:413
 [16] value(x::Number; result)
     @ C:\Users\slash\.julia\packages\JuMP\Ue9wj\src\variables.jl:2712
 [17] value(c::NonlinearConstraintRef; result)
     @ C:\Users\slash\.julia\packages\JuMP\Ue9wj\src\nlp.jl:619
 [18] value(con_ref::ConstraintRef{<:AbstractModel, <:MathOptInterface.ConstraintIndex}; result)
     @ C:\Users\slash\.julia\packages\JuMP\Ue9wj\src\constraints.jl:1302
 [19] value(Q::LinearAlgebra.Hermitian{V, Matrix{V}}; result) where V<:AbstractJuMPScalar
     @ C:\Users\slash\.julia\packages\JuMP\Ue9wj\src\sd.jl:887
 [20] value(v::GenericVariableRef{T}; result) where T
     @ C:\Users\slash\.julia\packages\JuMP\Ue9wj\src\variables.jl:2091
 [21] value(ex::GenericQuadExpr; result)
     @ C:\Users\slash\.julia\packages\JuMP\Ue9wj\src\quad_expr.jl:869
 [22] value(Q::AbstractArray{<:AbstractJuMPScalar}; result)
     @ C:\Users\slash\.julia\packages\JuMP\Ue9wj\src\variables.jl:2705
 [23] value(a::GenericNonlinearExpr; result)
     @ C:\Users\slash\.julia\packages\JuMP\Ue9wj\src\nlp_expr.jl:656
 [24] value(a::GenericAffExpr; result)
     @ C:\Users\slash\.julia\packages\JuMP\Ue9wj\src\aff_expr.jl:733

In [20]:
value(surface)

500.0

We will summarize the solution approach using a Julia function.

In [21]:
function farmer(factor::Float64 = 1.0, m::Model = Model(HiGHS.Optimizer))
    crops = ["Wheat", "Corn", "Suger beets"]
    ncrops = length(crops)

    @variable(m, x[1:ncrops] >= 0)
    @variable(m, y[1:2] >= 0)
    @variable(m, w[1:4] >= 0)

    costSeeding = [150, 230, 260]
    costs = [238, 210]
    prices = [170, 150, 36, 10]
    returns = factor*[2.5, 3, 20]

    @constraint(m, surface, sum(x[i] for i=1:3) <= 500)

    @constraint(m, wheatNeeds, returns[1]x[1]+y[1]-w[1] >= 200)
    @constraint(m, cornNeeds, returns[2]x[2]+y[2]-w[2] >= 240)
    @constraint(m, beetsProd, w[3]+w[4] <= returns[3]x[3])
    @constraint(m, beetsQuota, w[3] <= 6000)

    @objective(m,Min,sum(costSeeding[i]*x[i] for i=1:3) + sum(costs[i]*y[i] for i = 1:length(costs))
    - (sum(prices[i]*w[i] for i=1:length(prices))))
    
    status = optimize!(m)

    # println(value.(x))

    return m, status
end

farmer (generic function with 3 methods)

In [22]:
methods(farmer)

# 3 methods for generic function "farmer" from Main:
 [1] farmer(factor::Float64, m::Model)
     @ In[21]:1
 [2] farmer(factor::Float64)
     @ In[21]:1
 [3] farmer()
     @ In[21]:1

In [23]:
maverage, status = farmer()

Running HiGHS 1.11.0 (git hash: 364c83a51e): Copyright (c) 2025 HiGHS under MIT licence terms
LP   has 5 rows; 9 cols; 13 nonzeros
Coefficient ranges:
  Matrix [1e+00, 2e+01]
  Cost   [1e+01, 3e+02]
  Bound  [0e+00, 0e+00]
  RHS    [2e+02, 6e+03]
Presolving model
4 rows, 9 cols, 12 nonzeros  0s
3 rows, 8 cols, 10 nonzeros  0s
Dependent equations search running on 2 equations with time limit of 1000.00s
Dependent equations search removed 0 rows and 0 nonzeros in 0.00s (limit = 1000.00s)
3 rows, 7 cols, 9 nonzeros  0s
Presolve : Reductions: rows 3(-2); columns 7(-2); elements 9(-4)
Solving the presolved LP
Using EKK dual simplex solver - serial
  Iteration        Objective     Infeasibilities num(sum)
          0    -6.3999740210e+02 Ph1: 2(2); Du: 2(639.997) 0s
          4    -1.1860000000e+05 Pr: 0(0) 0s
Solving the original LP from the solution after postsolve
Model status        : Optimal
Simplex   iterations: 4
Objective value     : -1.1860000000e+05
P-D objective error :  0.0000000

(A JuMP Model
├ solver: HiGHS
├ objective_sense: MIN_SENSE
│ └ objective_function_type: AffExpr
├ num_variables: 9
├ num_constraints: 14
│ ├ AffExpr in MOI.GreaterThan{Float64}: 2
│ ├ AffExpr in MOI.LessThan{Float64}: 3
│ └ VariableRef in MOI.GreaterThan{Float64}: 9
└ Names registered in the model
  └ :beetsProd, :beetsQuota, :cornNeeds, :surface, :w, :wheatNeeds, :x, :y, nothing)

In [24]:
maverage

A JuMP Model
├ solver: HiGHS
├ objective_sense: MIN_SENSE
│ └ objective_function_type: AffExpr
├ num_variables: 9
├ num_constraints: 14
│ ├ AffExpr in MOI.GreaterThan{Float64}: 2
│ ├ AffExpr in MOI.LessThan{Float64}: 3
│ └ VariableRef in MOI.GreaterThan{Float64}: 9
└ Names registered in the model
  └ :beetsProd, :beetsQuota, :cornNeeds, :surface, :w, :wheatNeeds, :x, :y

In [25]:
print(maverage)

In [26]:
objective_value(maverage)

-118600.0

We can recover the decision variables with the method `all_variables`:

In [27]:
variables = all_variables(maverage)

9-element Vector{VariableRef}:
 x[1]
 x[2]
 x[3]
 y[1]
 y[2]
 w[1]
 w[2]
 w[3]
 w[4]

We can thus get the first-stage decisions.

In [28]:
value.(variables[1:3])

3-element Vector{Float64}:
 120.0
  80.0
 300.0

In [31]:
print(status)

nothing

## Good scenario

In [32]:
mgood, status = farmer(1.2)

Running HiGHS 1.11.0 (git hash: 364c83a51e): Copyright (c) 2025 HiGHS under MIT licence terms
LP   has 5 rows; 9 cols; 13 nonzeros
Coefficient ranges:
  Matrix [1e+00, 2e+01]
  Cost   [1e+01, 3e+02]
  Bound  [0e+00, 0e+00]
  RHS    [2e+02, 6e+03]
Presolving model
4 rows, 9 cols, 12 nonzeros  0s
3 rows, 8 cols, 10 nonzeros  0s
Dependent equations search running on 2 equations with time limit of 1000.00s
Dependent equations search removed 0 rows and 0 nonzeros in 0.00s (limit = 1000.00s)
3 rows, 7 cols, 9 nonzeros  0s
Presolve : Reductions: rows 3(-2); columns 7(-2); elements 9(-4)
Solving the presolved LP
Using EKK dual simplex solver - serial
  Iteration        Objective     Infeasibilities num(sum)
          0    -6.3999740210e+02 Ph1: 2(2); Du: 2(639.997) 0s
          4    -1.6766666667e+05 Pr: 0(0) 0s
Solving the original LP from the solution after postsolve
Model status        : Optimal
Simplex   iterations: 4
Objective value     : -1.6766666667e+05
P-D objective error :  8.6790488

(A JuMP Model
├ solver: HiGHS
├ objective_sense: MIN_SENSE
│ └ objective_function_type: AffExpr
├ num_variables: 9
├ num_constraints: 14
│ ├ AffExpr in MOI.GreaterThan{Float64}: 2
│ ├ AffExpr in MOI.LessThan{Float64}: 3
│ └ VariableRef in MOI.GreaterThan{Float64}: 9
└ Names registered in the model
  └ :beetsProd, :beetsQuota, :cornNeeds, :surface, :w, :wheatNeeds, :x, :y, nothing)

In [33]:
mgood

A JuMP Model
├ solver: HiGHS
├ objective_sense: MIN_SENSE
│ └ objective_function_type: AffExpr
├ num_variables: 9
├ num_constraints: 14
│ ├ AffExpr in MOI.GreaterThan{Float64}: 2
│ ├ AffExpr in MOI.LessThan{Float64}: 3
│ └ VariableRef in MOI.GreaterThan{Float64}: 9
└ Names registered in the model
  └ :beetsProd, :beetsQuota, :cornNeeds, :surface, :w, :wheatNeeds, :x, :y

In [34]:
objective_value(mgood)

-167666.6666666667

## Bad scenario

In [35]:
mbad, status = farmer(0.8)
#mbad, status = farmer(0.8, Model(solver = GurobiSolver()))

Running HiGHS 1.11.0 (git hash: 364c83a51e): Copyright (c) 2025 HiGHS under MIT licence terms
LP   has 5 rows; 9 cols; 13 nonzeros
Coefficient ranges:
  Matrix [1e+00, 2e+01]
  Cost   [1e+01, 3e+02]
  Bound  [0e+00, 0e+00]
  RHS    [2e+02, 6e+03]
Presolving model
4 rows, 9 cols, 12 nonzeros  0s
3 rows, 8 cols, 10 nonzeros  0s
Dependent equations search running on 2 equations with time limit of 1000.00s
Dependent equations search removed 0 rows and 0 nonzeros in 0.00s (limit = 1000.00s)
3 rows, 7 cols, 9 nonzeros  0s
Presolve : Reductions: rows 3(-2); columns 7(-2); elements 9(-4)
Solving the presolved LP
Using EKK dual simplex solver - serial
  Iteration        Objective     Infeasibilities num(sum)
          0    -4.6999813050e+02 Ph1: 2(2); Du: 2(469.998) 0s
          5    -5.9950000000e+04 Pr: 0(0) 0s
Solving the original LP from the solution after postsolve
Model status        : Optimal
Simplex   iterations: 5
Objective value     : -5.9950000000e+04
P-D objective error :  2.4273217

(A JuMP Model
├ solver: HiGHS
├ objective_sense: MIN_SENSE
│ └ objective_function_type: AffExpr
├ num_variables: 9
├ num_constraints: 14
│ ├ AffExpr in MOI.GreaterThan{Float64}: 2
│ ├ AffExpr in MOI.LessThan{Float64}: 3
│ └ VariableRef in MOI.GreaterThan{Float64}: 9
└ Names registered in the model
  └ :beetsProd, :beetsQuota, :cornNeeds, :surface, :w, :wheatNeeds, :x, :y, nothing)

In [36]:
objective_value(mbad)

-59950.0

## Stochastic program - extensive form

We have to combine the three scenarios. We will assume that each one has a probability equal to 1/3.

In [37]:
n = 3
p = [1/n for i = 1:n]

3-element Vector{Float64}:
 0.3333333333333333
 0.3333333333333333
 0.3333333333333333

We will start with a naive implementation.

In [38]:
function farmerStoch()
    m = Model(HiGHS.Optimizer)

    crops = ["Wheat", "Corn", "Suger beets"]
    ncrops = length(crops)

    @variable(m,x[1:ncrops] >= 0)

    @variable(m,ya[1:2] >= 0)
    @variable(m,wa[1:4] >= 0)

    @variable(m,yg[1:2] >= 0)
    @variable(m,wg[1:4] >= 0)

    @variable(m,yb[1:2] >= 0)
    @variable(m,wb[1:4] >= 0)

    costSeeding = [150, 230, 260]
    costs = [238, 210]
    prices = [170, 150, 36, 10]
    returnsA = [2.5, 3, 20]
    returnsG = 1.2*returnsA
    returnsB = 0.8*returnsA

    @constraint(m, surface, sum(x[i] for i=1:3) <= 500)

    @constraint(m, wheatNA, returnsA[1]x[1]+ya[1]-wa[1] >= 200)
    @constraint(m, cornNA, returnsA[2]x[2]+ya[2]-wa[2] >= 240)
    @constraint(m, beetsPA, wa[3]+wa[4] <= returnsA[3]x[3])
    @constraint(m, beetsQA, wa[3] <= 6000)

    @constraint(m, wheatNG, returnsG[1]x[1]+yg[1]-wg[1] >= 200)
    @constraint(m, cornNG, returnsG[2]x[2]+yg[2]-wg[2] >= 240)
    @constraint(m, beetsPG, wg[3]+wg[4] <= returnsG[3]x[3])
    @constraint(m, beetsQG, wg[3] <= 6000)

    @constraint(m, wheatNB, returnsB[1]x[1]+yb[1]-wb[1] >= 200)
    @constraint(m, cornNB, returnsB[2]x[2]+yb[2]-wb[2] >= 240)
    @constraint(m, beetsPB, wb[3]+wb[4] <= returnsB[3]x[3])
    @constraint(m, beetsQB, wb[3] <= 6000)

    @objective(m, Min, sum(costSeeding[i]*x[i] for i=1:3)
        + 1/3*sum(costs[i]*ya[i] for i = 1:length(costs))
        - 1/3*sum(prices[i]*wa[i] for i=1:length(prices))
        + 1/3*sum(costs[i]*yg[i] for i = 1:length(costs))
        - 1/3*sum(prices[i]*wg[i] for i=1:length(prices))
        + 1/3*sum(costs[i]*yb[i] for i = 1:length(costs))
        - 1/3*sum(prices[i]*wb[i] for i=1:length(prices)))
    
    status = optimize!(m)

    return m, status
end

farmerStoch (generic function with 1 method)

In [39]:
mstoch, status = farmerStoch()

Running HiGHS 1.11.0 (git hash: 364c83a51e): Copyright (c) 2025 HiGHS under MIT licence terms
LP   has 13 rows; 21 cols; 33 nonzeros
Coefficient ranges:
  Matrix [1e+00, 2e+01]
  Cost   [3e+00, 3e+02]
  Bound  [0e+00, 0e+00]
  RHS    [2e+02, 6e+03]
Presolving model
10 rows, 21 cols, 30 nonzeros  0s
9 rows, 20 cols, 30 nonzeros  0s
Presolve : Reductions: rows 9(-4); columns 20(-1); elements 30(-3)
Solving the presolved LP
Using EKK dual simplex solver - serial
  Iteration        Objective     Infeasibilities num(sum)
          0    -3.1831206476e+02 Ph1: 8(8); Du: 8(318.312) 0s
         15    -1.0839000000e+05 Pr: 0(0) 0s
Solving the original LP from the solution after postsolve
Model status        : Optimal
Simplex   iterations: 15
Objective value     : -1.0839000000e+05
P-D objective error :  6.7127263129e-17
HiGHS run time      :          0.00


(A JuMP Model
├ solver: HiGHS
├ objective_sense: MIN_SENSE
│ └ objective_function_type: AffExpr
├ num_variables: 21
├ num_constraints: 34
│ ├ AffExpr in MOI.GreaterThan{Float64}: 6
│ ├ AffExpr in MOI.LessThan{Float64}: 7
│ └ VariableRef in MOI.GreaterThan{Float64}: 21
└ Names registered in the model
  └ :beetsPA, :beetsPB, :beetsPG, :beetsQA, :beetsQB, :beetsQG, :cornNA, :cornNB, :cornNG, :surface, :wa, :wb, :wg, :wheatNA, :wheatNB, :wheatNG, :x, :ya, :yb, :yg, nothing)

In [40]:
print(mstoch)

In [41]:
valStoch = objective_value(mstoch)

-108390.0

## Expected value of the perfect information

In [42]:
values = [objective_value(maverage), objective_value(mgood), objective_value(mbad)]

3-element Vector{Float64}:
 -118600.0
 -167666.6666666667
  -59950.0

Let's compute the expected revenue under perfect information, simply obtained as the dot product between the probabilities vector and the optimal values vector.

In [43]:
valPerfect = p'*values

-115405.55555555555

In [44]:
using LinearAlgebra

dot(p, values)

-115405.55555555555

In [45]:
methods(dot)

# 61 methods for generic function "dot" from LinearAlgebra:
  [1] dot(x::BitVector, y::BitVector)
     @ C:\Users\slash\.julia\juliaup\julia-1.11.6+0.x64.w64.mingw32\share\julia\stdlib\v1.11\LinearAlgebra\src\bitarray.jl:3
  [2] dot(x::Vector{T}, rx::AbstractRange{TI}, y::Vector{T}, ry::AbstractRange{TI}) where {T<:Union{ComplexF64, ComplexF32}, TI<:Integer}
     @ C:\Users\slash\.julia\juliaup\julia-1.11.6+0.x64.w64.mingw32\share\julia\stdlib\v1.11\LinearAlgebra\src\matmul.jl:34
  [3] dot(x::Vector{T}, rx::AbstractRange{TI}, y::Vector{T}, ry::AbstractRange{TI}) where {T<:Union{Float32, Float64}, TI<:Integer}
     @ C:\Users\slash\.julia\juliaup\julia-1.11.6+0.x64.w64.mingw32\share\julia\stdlib\v1.11\LinearAlgebra\src\matmul.jl:21
  [4] dot(x::Adjoint{<:Union{Real, Complex}}, y::Adjoint{<:Union{Real, Complex}})
     @ C:\Users\slash\.julia\juliaup\julia-1.11.6+0.x64.w64.mingw32\share\julia\stdlib\v1.11\LinearAlgebra\src\generic.jl:901
  [5] dot(x::Symmetric{var"#s38", S} where {var"#s38"<:MutableArithmetics.AbstractMutable, S<:(AbstractMatrix{<:var"#s38"})}, y::Symmetric{var"#s38", S} where {var"#s38"<:MutableArithmetics.AbstractMutable, S<:(AbstractMatrix{<:var"#s38"})})
     @ MutableArithmetics C:\Users\slash\.julia\packages\MutableArithmetics\tNSBd\src\dispatch.jl:69
  [6] dot(x::Symmetric, y::Symmetric{var"#s38", S} where {var"#s38"<:MutableArithmetics.AbstractMutable, S<:(AbstractMatrix{<:var"#s38"})})
     @ MutableArithmetics C:\Users\slash\.julia\packages\MutableArithmetics\tNSBd\src\dispatch.jl:67
  [7] dot(x::Symmetric{var"#s38", S} where {var"#s38"<:MutableArithmetics.AbstractMutable, S<:(AbstractMatrix{<:var"#s38"})}, y::Symmetric)
     @ MutableArithmetics C:\Users\slash\.julia\packages\MutableArithmetics\tNSBd\src\dispatch.jl:68
  [8] dot(A::Symmetric, B::Symmetric)
     @ C:\Users\slash\.julia\juliaup\julia-1.11.6+0.x64.w64.mingw32\share\julia\stdlib\v1.11\LinearAlgebra\src\symmetric.jl:485
  [9] dot(x::SparseArrays.SparseVector, A::SparseArrays.AbstractSparseMatrixCSC, y::SparseArrays.SparseVector)
     @ SparseArrays C:\Users\slash\.julia\juliaup\julia-1.11.6+0.x64.w64.mingw32\share\julia\stdlib\v1.11\SparseArrays\src\linalg.jl:568
 [10] dot(x::SparseArrays.SparseVector, A::Union{Hermitian{var"#s393", var"#s392"}, Hermitian{Complex{var"#s393"}, var"#s392"}, Symmetric{var"#s393", var"#s392"}} where {var"#s393", var"#s392"<:SparseArrays.AbstractSparseMatrixCSC}, y::SparseArrays.SparseVector)
     @ SparseArrays C:\Users\slash\.julia\juliaup\julia-1.11.6+0.x64.w64.mingw32\share\julia\stdlib\v1.11\SparseArrays\src\linalg.jl:1149
 [11] dot(x::AbstractVector, A::Union{Hermitian{var"#s5052", var"#s5051"}, Symmetric{var"#s5052", var"#s5051"}, Symmetric{Complex{var"#s5052"}, var"#s5051"}} where {var"#s5052"<:Real, var"#s5051"<:Diagonal}, y::AbstractVector)
     @ C:\Users\slash\.julia\juliaup\julia-1.11.6+0.x64.w64.mingw32\share\julia\stdlib\v1.11\LinearAlgebra\src\special.jl:326
 [12] dot(x::Transpose, y::Transpose)
     @ C:\Users\slash\.julia\juliaup\julia-1.11.6+0.x64.w64.mingw32\share\julia\stdlib\v1.11\LinearAlgebra\src\generic.jl:904
 [13] dot(α::T, func::Union{MathOptInterface.ScalarAffineFunction{T}, MathOptInterface.ScalarQuadraticFunction{T}, MathOptInterface.VectorAffineFunction{T}, MathOptInterface.VectorQuadraticFunction{T}}) where T<:Number
     @ MathOptInterface.Utilities C:\Users\slash\.julia\packages\MathOptInterface\QyCeQ\src\Utilities\functions.jl:1867
 [14] dot(x::Number, y::Number)
     @ C:\Users\slash\.julia\juliaup\julia-1.11.6+0.x64.w64.mingw32\share\julia\stdlib\v1.11\LinearAlgebra\src\generic.jl:884
 [15] dot(A::SparseArrays.AbstractSparseMatrixCSC{T1, S1}, B::SparseArrays.AbstractSparseMatrixCSC{T2, S2}) where {T1, T2, S1, S2}
     @ SparseArrays C:\Users\slash\.julia\juliaup\julia-1.11.6+0.x64.w64.mingw32\share\julia\stdlib\v1.11\SparseArrays\src\linalg.jl:516
 [16] dot(A::SparseArrays.AbstractSparseMatrixCSC, B::Union{SubArray{<:Any, 2, Union{StridedMatrix, BitMatrix, SparseArrays.Abstr

Expected value of perfect information:

In [46]:
valPerfect - valStoch

-7015.5555555555475

## Value of the stochastic solution

We have to fix the first stage decision. We define the second stage problem.

In [47]:
function secondStage(x, factor::Float64 = 1.0)
    m = Model(HiGHS.Optimizer)
    
    @variable(m,y[1:2] >= 0)   # purchases
    @variable(m,w[1:4] >= 0)   # sales

    costs = [238, 210]
    prices = [170, 150, 36, 10]
    returns = (factor*[2.5, 3, 20]).*x
    z = [200 - returns[1], 240 - returns[2], returns[3]]
    
    @constraint(m, wheatNeeds, y[1]-w[1] >= z[1])
    @constraint(m, cornNeeds, y[2]-w[2] >= z[2])
    @constraint(m, beetsProd, w[3]+w[4] <= z[3])
    @constraint(m, beetsQuota, w[3] <= 6000)

    @objective(m,Min, sum(costs[i]*y[i] for i = 1:length(costs)) - (sum(prices[i]*w[i] for i=1:length(prices))))
    
    status = optimize!(m)
    
    return m, status
end

secondStage (generic function with 2 methods)

In [48]:
x = [120, 80, 300]

3-element Vector{Int64}:
 120
  80
 300

In [49]:
msecond, status = secondStage(x)

Running HiGHS 1.11.0 (git hash: 364c83a51e): Copyright (c) 2025 HiGHS under MIT licence terms
LP   has 4 rows; 6 cols; 7 nonzeros
Coefficient ranges:
  Matrix [1e+00, 1e+00]
  Cost   [1e+01, 2e+02]
  Bound  [0e+00, 0e+00]
  RHS    [1e+02, 6e+03]
Presolving model
2 rows, 3 cols, 3 nonzeros  0s
0 rows, 0 cols, 0 nonzeros  0s
Presolve : Reductions: rows 0(-4); columns 0(-6); elements 0(-7) - Reduced to empty
Solving the original LP from the solution after postsolve
Using EKK primal simplex solver
  Iteration        Objective     Infeasibilities num(sum)
          0    -2.3300006374e+05 Pr: 0(0); Du: 1(10) 0s
          1    -2.3300000000e+05 Pr: 0(0) 0s
Required 1 simplex iterations after postsolve
Model status        : Optimal
Simplex   iterations: 1
Objective value     : -2.3300000000e+05
P-D objective error :  0.0000000000e+00
HiGHS run time      :          0.01


(A JuMP Model
├ solver: HiGHS
├ objective_sense: MIN_SENSE
│ └ objective_function_type: AffExpr
├ num_variables: 6
├ num_constraints: 10
│ ├ AffExpr in MOI.GreaterThan{Float64}: 2
│ ├ AffExpr in MOI.LessThan{Float64}: 2
│ └ VariableRef in MOI.GreaterThan{Float64}: 6
└ Names registered in the model
  └ :beetsProd, :beetsQuota, :cornNeeds, :w, :wheatNeeds, :y, nothing)

In [50]:
print(msecond)

In [51]:
vaverage = objective_value(msecond)

-233000.0

In [52]:
msecond, status = secondStage(x, 1.2)

Running HiGHS 1.11.0 (git hash: 364c83a51e): Copyright (c) 2025 HiGHS under MIT licence terms
LP   has 4 rows; 6 cols; 7 nonzeros
Coefficient ranges:
  Matrix [1e+00, 1e+00]
  Cost   [1e+01, 2e+02]
  Bound  [0e+00, 0e+00]
  RHS    [5e+01, 7e+03]
Presolving model
3 rows, 6 cols, 6 nonzeros  0s
0 rows, 0 cols, 0 nonzeros  0s
Presolve : Reductions: rows 0(-4); columns 0(-6); elements 0(-7) - Reduced to empty
Solving the original LP from the solution after postsolve
Model status        : Optimal
Objective value     : -2.6240000000e+05
P-D objective error :  0.0000000000e+00
HiGHS run time      :          0.00


(A JuMP Model
├ solver: HiGHS
├ objective_sense: MIN_SENSE
│ └ objective_function_type: AffExpr
├ num_variables: 6
├ num_constraints: 10
│ ├ AffExpr in MOI.GreaterThan{Float64}: 2
│ ├ AffExpr in MOI.LessThan{Float64}: 2
│ └ VariableRef in MOI.GreaterThan{Float64}: 6
└ Names registered in the model
  └ :beetsProd, :beetsQuota, :cornNeeds, :w, :wheatNeeds, :y, nothing)

In [53]:
vgood = objective_value(msecond)

-262400.0

In [54]:
msecond, status = secondStage(x, 0.8)

Running HiGHS 1.11.0 (git hash: 364c83a51e): Copyright (c) 2025 HiGHS under MIT licence terms
LP   has 4 rows; 6 cols; 7 nonzeros
Coefficient ranges:
  Matrix [1e+00, 1e+00]
  Cost   [1e+01, 2e+02]
  Bound  [0e+00, 0e+00]
  RHS    [4e+01, 6e+03]
Presolving model
2 rows, 3 cols, 3 nonzeros  0s
0 rows, 0 cols, 0 nonzeros  0s
Presolve : Reductions: rows 0(-4); columns 0(-6); elements 0(-7) - Reduced to empty
Solving the original LP from the solution after postsolve
Model status        : Optimal
Objective value     : -1.6952000000e+05
P-D objective error :  0.0000000000e+00
HiGHS run time      :          0.00


(A JuMP Model
├ solver: HiGHS
├ objective_sense: MIN_SENSE
│ └ objective_function_type: AffExpr
├ num_variables: 6
├ num_constraints: 10
│ ├ AffExpr in MOI.GreaterThan{Float64}: 2
│ ├ AffExpr in MOI.LessThan{Float64}: 2
│ └ VariableRef in MOI.GreaterThan{Float64}: 6
└ Names registered in the model
  └ :beetsProd, :beetsQuota, :cornNeeds, :w, :wheatNeeds, :y, nothing)

In [55]:
vbad = objective_value(msecond)

-169520.0

Expected revenue

In [56]:
er = costSeeding'*x+p'*[vaverage, vgood, vbad]

-107239.99999999997

Value of the stochastic solution

In [57]:
vss = valStoch - er

-1150.000000000029